In [65]:
import shutil
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

### DOWNLOAD PHOTO

In [2]:
url = 'https://c1.staticflickr.com/9/8094/8534020390_ee8b1b9e67_h.jpg'
file_name = 'data/images/pro/test.jpg'
r = requests.get(url, stream=True)
with open(file_name, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)
del r

### SEARACH PHOTOS

In [51]:
landscape_search_url = 'https://www.flickr.com/search/'
landscape_search_kwds = {'dimension_search_mode': 'min&height=1024&width=1024',
                         'media': 'photos',
                         'advanced': '1',
                         'text': 'landscape pro'}
headers = {'user-agent': 'Mozilla/5.0'}
r = requests.get(landscape_search_url, params=landscape_search_kwds, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

In [52]:
soup.select('div.content')

[]

In [53]:
soup.find_all('div', class_='photo-list-photo-interaction')

[]

### SEARCH PEOPLE

In [57]:
people_search_url = 'https://www.flickr.com/photos/12037949754@N01/'
# people_search_kwds = {'username': 'pro'}
headers = {'user-agent': 'Mozilla/5.0'}
r = requests.get(landscape_search_url)
soup = BeautifulSoup(r.text, 'html.parser')

### FLICKR API PHOTO SEARCH

In [10]:
keys_list = []
keys = {}
with open('flickr.keys') as f_open:
    for line in f_open:
        keys_list.append(line.strip('\n'))

In [14]:
for key in keys_list:
    pair = key.split(":")
    keys[pair[0]] = pair[1]

In [15]:
keys

{'Key': '933b26674bf1f7a00109f2f8eda30b52', 'Secret': '1bd86a22b5ea95bf'}

In [54]:
def search_flickr(search_tag_term, page_num=1):
    search_url = 'https://api.flickr.com/services/rest/'
    search_kwds = {'method':'flickr.photos.search',
                   'api_key':keys['Key'],
                   'tags':search_tag_term,
                   'sort':'relavance',
                   'content_type':'1',
                   'per_page':'500',
                   'page':page_num}
    r = requests.get(search_url, params=search_kwds)
    soup = BeautifulSoup(r.text, 'lxml')
    return soup, r.status_code

In [56]:
search_tag_term = 'landscape'
soup, status= search_flickr(search_tag_term)
total_pages = int(soup.photos.get('pages'))
print 'Status: {}'.format(status)

Stats: 200


In [60]:
pictures = []
for i in range(1, total_pages+1):
    soup, status = search_flickr(search_tag_term, page_num=i)
    if int(status) != 200:
        print "Error on Page {}: Status Code {}".format(i, status)
        print "Print soup to determine problem."
        break
    if i == 1:
        print "Begining Search for '{}': {} Pages Found".format(search_tag_term, total_pages)
    pictures.extend(soup.findAll('photo'))
    if i%50==0:
        print "{} Pages Complete".format(i)
    if i == total_pages:
        print "Search Complete"
        print "-"*30

50 Pages Complete
100 Pages Complete
150 Pages Complete
200 Pages Complete
250 Pages Complete
300 Pages Complete
350 Pages Complete
400 Pages Complete
450 Pages Complete
500 Pages Complete
550 Pages Complete
600 Pages Complete
650 Pages Complete
700 Pages Complete
750 Pages Complete
800 Pages Complete
850 Pages Complete
900 Pages Complete
950 Pages Complete


In [61]:
len(pictures)

495965

In [64]:
for picture in pictures[:1]:
    print picture.get('id')

27797327313


In [33]:
soup.findAll("photo")[0]

<photo farm="8" id="27796858203" isfamily="0" isfriend="0" ispublic="1" owner="69573851@N06" secret="fa18e51459" server="7513" title="COLLARED FINCH-BILLED BULBUL"></photo>

In [5]:
url = 'https://farm{farm_id}.staticflickr.com/{server_id}/{ID}_{secret}_{mstzb}.jpg'
url_full = url.format(farm_id=farm_id, server_id=server_id, ID=ID, secret=secret, mstzb=mstzb)

NameError: name 'farm_id' is not defined